In [1]:
# %% [markdown]
# # RUL Prediction on NASA C-MAPSS FD001
#
# Compare four approaches:
# 1. LSTM  
# 2. CNNTransformer  
# 3. BiLSTM + Attention  
# 4. TCN + Attention  
#
# Hyperparameters at top for easy fine-tuning.  
# Timers and per-epoch summaries (loss, val_loss, mae, val_mae, mse, val_mse).

# %% [code]
# 0. Imports & Globals
%pip install -q torch torchvision torchaudio scikit-learn matplotlib
import os, time, random
import numpy as np, pandas as pd
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
SEQ_LEN       = 50
BATCH_SIZE    = 64  #64
LR            = 1e-3
MAX_LR        = 3e-3
EPOCHS_LSTM   = 50
EPOCHS_TF     = 50
EPOCHS_BILSTM = 50
EPOCHS_TCN    = 50
WEIGHT_DECAY  = 1e-4
DROPOUT       = 0.2

# Loss functions
mae_loss = nn.L1Loss(reduction='mean')
mse_loss = nn.MSELoss(reduction='mean')
PRIMARY_LOSS = mse_loss  # switch to mae_loss if desired

In [3]:
# %% [code]
# 1. Load & preprocess data
DATA_DIR = '/kaggle/input/nasa-cmaps/CMaps'
cols = ['unit','cycle'] + [f'op_setting_{i}' for i in range(1,4)] + [f'sensor_{i}' for i in range(1,22)]
train_df = pd.read_csv(f'{DATA_DIR}/train_FD001.txt', sep='\s+', names=cols)
test_df  = pd.read_csv(f'{DATA_DIR}/test_FD001.txt',  sep='\s+', names=cols)
rul_df   = pd.read_csv(f'{DATA_DIR}/RUL_FD001.txt',   names=['RUL'])

# Compute RUL for train
max_cycle = train_df.groupby('unit')['cycle'].max().to_dict()
train_df['RUL'] = train_df.apply(lambda r: max_cycle[r.unit] - r.cycle, axis=1)

# Normalize sensors
SENSORS = [c for c in train_df if c.startswith('sensor_')]
scaler  = MinMaxScaler()
train_df[SENSORS] = scaler.fit_transform(train_df[SENSORS])
test_df[SENSORS]  = scaler.transform(test_df[SENSORS])

# Hold-out split by unit
units = train_df['unit'].unique()
valid_units = np.random.choice(units, size=int(0.2*len(units)), replace=False)
train_split = train_df[~train_df.unit.isin(valid_units)].copy()
valid_split = train_df[ train_df.unit.isin(valid_units)].copy()

In [4]:
# %% [code]
# 2. Dataset & DataLoader
class SequenceDataset(Dataset):
    def __init__(self, df, seq_len, is_train=True, rul=None):
        self.seq_len = seq_len
        self.data = []
        cols = SENSORS + ['op_setting_1','op_setting_2','op_setting_3']
        for u in df.unit.unique():
            vals = df[df.unit==u].sort_values('cycle')[cols].values
            if is_train:
                labels = df[df.unit==u].sort_values('cycle')['RUL'].values
                for i in range(len(vals)-seq_len+1):
                    self.data.append((vals[i:i+seq_len], labels[i+seq_len-1]))
            else:
                r_val = rul.loc[u-1,'RUL']
                for i in range(len(vals)-seq_len+1):
                    self.data.append((vals[i:i+seq_len], r_val))
    def __len__(self): return len(self.data)
    def __getitem__(self, idx):
        x,y = self.data[idx]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

train_ds = SequenceDataset(train_split, SEQ_LEN, is_train=True)
valid_ds = SequenceDataset(valid_split, SEQ_LEN, is_train=True)
train_loader = DataLoader(train_ds, BATCH_SIZE, shuffle=True,  num_workers=0)
valid_loader = DataLoader(valid_ds, BATCH_SIZE, shuffle=False, num_workers=0)

In [5]:
# %% [markdown]
# ## 3. LSTM

# %% [code]
class LSTMModel(nn.Module):
    def __init__(self, in_feats, hidden=128, layers=2, dropout=0.3):
        super().__init__()
        self.lstm = nn.LSTM(in_feats, hidden, layers,
                            batch_first=True, dropout=dropout)
        self.fc   = nn.Sequential(
            nn.Linear(hidden, 64),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64,1)
        )
    def forward(self, x):
        out,_ = self.lstm(x)
        return self.fc(out[:,-1,:]).squeeze(1)

model_lstm = LSTMModel(len(SENSORS)+3).to(DEVICE)
opt_lstm   = AdamW(model_lstm.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
sched_lstm = OneCycleLR(opt_lstm, MAX_LR, steps_per_epoch=len(train_loader), epochs=EPOCHS_LSTM)

# Training loop template
def train_model(model, optimizer, scheduler, epochs, name):
    best = float('inf')
    start = time.time()
    for epoch in range(1, epochs+1):
        model.train(); tm_mae=tm_mse=0
        for xb,yb in train_loader:
            xb,yb = xb.to(DEVICE), yb.to(DEVICE)
            pred = model(xb)
            loss = PRIMARY_LOSS(pred, yb)
            optimizer.zero_grad(); loss.backward(); optimizer.step(); scheduler.step()
            tm_mae += mae_loss(pred, yb).item()*xb.size(0)
            tm_mse += mse_loss(pred, yb).item()*xb.size(0)
        tm_mae /= len(train_ds); tm_mse /= len(train_ds)

        model.eval(); vm_mae=vm_mse=0
        with torch.no_grad():
            for xb,yb in valid_loader:
                xb,yb = xb.to(DEVICE), yb.to(DEVICE)
                pred = model(xb)
                vm_mae += mae_loss(pred, yb).item()*xb.size(0)
                vm_mse += mse_loss(pred, yb).item()*xb.size(0)
        vm_mae /= len(valid_ds); vm_mse /= len(valid_ds)

        loss_tr = tm_mae if PRIMARY_LOSS is mae_loss else tm_mse
        loss_val = vm_mae if PRIMARY_LOSS is mae_loss else vm_mse

        print(f"[{name}] Epoch {epoch:03d} "
              f"- loss: {loss_tr:.4f} - mae: {tm_mae:.4f} - mse: {tm_mse:.4f} "
              f"- val_loss: {loss_val:.4f} - val_mae: {vm_mae:.4f} - val_mse: {vm_mse:.4f}")

    print(f"[{name}] Total time: {(time.time()-start)/60:.2f} min\n")

train_model(model_lstm, opt_lstm, sched_lstm, EPOCHS_LSTM, "LSTM")

[LSTM] Epoch 001 - loss: 10043.5531 - mae: 81.6639 - mse: 10043.5531 - val_loss: 7853.9528 - val_mae: 71.7463 - val_mse: 7853.9528
[LSTM] Epoch 002 - loss: 7148.4570 - mae: 65.2597 - mse: 7148.4570 - val_loss: 4359.8506 - val_mae: 51.4305 - val_mse: 4359.8506
[LSTM] Epoch 003 - loss: 4006.6402 - mae: 48.8840 - mse: 4006.6402 - val_loss: 2840.6854 - val_mae: 44.3466 - val_mse: 2840.6854
[LSTM] Epoch 004 - loss: 3507.4234 - mae: 47.3566 - mse: 3507.4234 - val_loss: 2842.1057 - val_mae: 44.5343 - val_mse: 2842.1057
[LSTM] Epoch 005 - loss: 3515.4413 - mae: 47.4696 - mse: 3515.4413 - val_loss: 2846.8251 - val_mae: 44.6457 - val_mse: 2846.8251
[LSTM] Epoch 006 - loss: 3494.9841 - mae: 47.3770 - mse: 3494.9841 - val_loss: 2841.0607 - val_mae: 44.5000 - val_mse: 2841.0607
[LSTM] Epoch 007 - loss: 3514.1510 - mae: 47.4461 - mse: 3514.1510 - val_loss: 2840.1912 - val_mae: 44.4576 - val_mse: 2840.1912
[LSTM] Epoch 008 - loss: 3517.6554 - mae: 47.5892 - mse: 3517.6554 - val_loss: 2857.7456 - val_

In [6]:
# %% [markdown]
# ## 4. CNN-Transformer

# %% [code]
class CNNTransformerRUL(nn.Module):
    def __init__(self, in_feats, cnn_ch=64, heads=4, layers=2, ff=128, dropout=DROPOUT):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(in_feats, cnn_ch, 3, padding=1), nn.ReLU(),
            nn.Conv1d(cnn_ch, cnn_ch, 3, padding=1),   nn.ReLU()
        )
        self.pos = nn.Parameter(torch.randn(1, SEQ_LEN, cnn_ch))
        enc = nn.TransformerEncoderLayer(d_model=cnn_ch, nhead=heads,
                                         dim_feedforward=ff, dropout=dropout,
                                         batch_first=True)
        self.encoder = nn.TransformerEncoder(enc, num_layers=layers)
        self.head    = nn.Sequential(nn.Linear(cnn_ch, ff), nn.ReLU(),
                                     nn.Dropout(dropout), nn.Linear(ff,1))
    def forward(self, x):
        x = x.transpose(1,2)
        x = self.cnn(x).transpose(1,2) + self.pos
        out = self.encoder(x)
        return self.head(out[:,-1,:]).squeeze(1)

model_tf = CNNTransformerRUL(len(SENSORS)+3).to(DEVICE)
opt_tf    = AdamW(model_tf.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
sched_tf  = OneCycleLR(opt_tf, MAX_LR, steps_per_epoch=len(train_loader), epochs=EPOCHS_TF)

train_model(model_tf, opt_tf, sched_tf, EPOCHS_TF, "CNN-Transformer")

[CNN-Transformer] Epoch 001 - loss: 10156.1445 - mae: 82.3092 - mse: 10156.1445 - val_loss: 8383.2184 - val_mae: 74.9078 - val_mse: 8383.2184
[CNN-Transformer] Epoch 002 - loss: 7746.3377 - mae: 68.6188 - mse: 7746.3377 - val_loss: 4537.4020 - val_mae: 52.4190 - val_mse: 4537.4020
[CNN-Transformer] Epoch 003 - loss: 3912.2156 - mae: 48.5354 - mse: 3912.2156 - val_loss: 2816.9415 - val_mae: 44.0928 - val_mse: 2816.9415
[CNN-Transformer] Epoch 004 - loss: 1910.3890 - mae: 30.3976 - mse: 1910.3890 - val_loss: 515.9893 - val_mae: 16.7823 - val_mse: 515.9893
[CNN-Transformer] Epoch 005 - loss: 930.9834 - mae: 20.7834 - mse: 930.9834 - val_loss: 374.0522 - val_mae: 14.3646 - val_mse: 374.0522
[CNN-Transformer] Epoch 006 - loss: 870.9004 - mae: 20.0680 - mse: 870.9004 - val_loss: 403.8412 - val_mae: 15.1947 - val_mse: 403.8412
[CNN-Transformer] Epoch 007 - loss: 833.9436 - mae: 19.6464 - mse: 833.9436 - val_loss: 481.0009 - val_mae: 15.9303 - val_mse: 481.0009
[CNN-Transformer] Epoch 008 - lo

In [7]:
# %% [markdown]
# ## 5. BiLSTM + Attention

# %% [code]
class BiLSTMAttn(nn.Module):
    def __init__(self, in_feats, hidden=128, layers=1, dropout=DROPOUT):
        super().__init__()
        self.lstm = nn.LSTM(in_feats, hidden, layers,
                            batch_first=True, bidirectional=True, dropout=dropout)
        self.attn = nn.Linear(hidden*2, 1)
        self.head = nn.Sequential(nn.Linear(hidden*2,64),
                                  nn.ReLU(), nn.Dropout(dropout),
                                  nn.Linear(64,1))
    def forward(self, x):
        out,_ = self.lstm(x)
        w     = torch.softmax(self.attn(out), dim=1)
        ctx   = (w * out).sum(dim=1)
        return self.head(ctx).squeeze(1)

model_bilstm  = BiLSTMAttn(len(SENSORS)+3).to(DEVICE)
opt_bilstm   = AdamW(model_bilstm.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
sched_bilstm = OneCycleLR(opt_bilstm, MAX_LR, steps_per_epoch=len(train_loader), epochs=EPOCHS_BILSTM)

train_model(model_bilstm, opt_bilstm, sched_bilstm, EPOCHS_BILSTM, "BiLSTM+Attn")

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


[BiLSTM+Attn] Epoch 001 - loss: 10204.9339 - mae: 82.5370 - mse: 10204.9339 - val_loss: 7660.6106 - val_mae: 70.5877 - val_mse: 7660.6106
[BiLSTM+Attn] Epoch 002 - loss: 5729.7540 - mae: 57.6261 - mse: 5729.7540 - val_loss: 3010.9899 - val_mae: 44.5724 - val_mse: 3010.9899
[BiLSTM+Attn] Epoch 003 - loss: 3470.1135 - mae: 46.8085 - mse: 3470.1135 - val_loss: 2682.7261 - val_mae: 43.1283 - val_mse: 2682.7261
[BiLSTM+Attn] Epoch 004 - loss: 2839.0273 - mae: 41.8400 - mse: 2839.0273 - val_loss: 1467.5434 - val_mae: 29.9685 - val_mse: 1467.5434
[BiLSTM+Attn] Epoch 005 - loss: 2554.5580 - mae: 39.3502 - mse: 2554.5580 - val_loss: 1614.6310 - val_mae: 32.0958 - val_mse: 1614.6310
[BiLSTM+Attn] Epoch 006 - loss: 1672.4661 - mae: 30.1452 - mse: 1672.4661 - val_loss: 801.0459 - val_mae: 20.3119 - val_mse: 801.0459
[BiLSTM+Attn] Epoch 007 - loss: 1332.1746 - mae: 26.0459 - mse: 1332.1746 - val_loss: 490.2701 - val_mae: 15.3210 - val_mse: 490.2701
[BiLSTM+Attn] Epoch 008 - loss: 1041.7977 - mae: 2

In [8]:
# %% [markdown]
# ## 6. TCN + Attention

# %% [code]
from torch.nn.utils import weight_norm
class TCNBlock(nn.Module):
    def __init__(self, in_ch, out_ch, kernel=3, dilation=1):
        super().__init__()
        self.conv = weight_norm(nn.Conv1d(in_ch, out_ch, kernel,
                                          padding=(kernel-1)//2*dilation,
                                          dilation=dilation))
        self.relu = nn.ReLU()
        self.res  = nn.Conv1d(in_ch, out_ch,1) if in_ch!=out_ch else nn.Identity()
    def forward(self, x):
        y = self.relu(self.conv(x)); return y + self.res(x)

class TCN_Attn(nn.Module):
    def __init__(self, in_feats, ch=64, levels=5, heads=4, dropout=DROPOUT):
        super().__init__()
        blocks = [TCNBlock(in_feats if i==0 else ch, ch, dilation=2**i) for i in range(levels)]
        self.tcn  = nn.Sequential(*blocks)
        self.attn = nn.MultiheadAttention(ch, heads, dropout=dropout, batch_first=True)
        self.head = nn.Sequential(nn.Linear(ch,64), nn.ReLU(),
                                  nn.Dropout(dropout), nn.Linear(64,1))
    def forward(self, x):
        x = x.transpose(1,2)
        h = self.tcn(x).transpose(1,2)
        att,_ = self.attn(h, h, h)
        ctx   = att.mean(dim=1)
        return self.head(ctx).squeeze(1)

model_tcn = TCN_Attn(len(SENSORS)+3).to(DEVICE)
opt_tcn   = AdamW(model_tcn.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
sched_tcn = OneCycleLR(opt_tcn, MAX_LR, steps_per_epoch=len(train_loader), epochs=EPOCHS_TCN)

train_model(model_tcn, opt_tcn, sched_tcn, EPOCHS_TCN, "TCN+Attn")

/usr/local/lib/python3.11/dist-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


[TCN+Attn] Epoch 001 - loss: 4703.2997 - mae: 53.4510 - mse: 4703.2997 - val_loss: 2836.5502 - val_mae: 44.4758 - val_mse: 2836.5502
[TCN+Attn] Epoch 002 - loss: 3576.5672 - mae: 47.7838 - mse: 3576.5672 - val_loss: 2836.5164 - val_mae: 44.6058 - val_mse: 2836.5164
[TCN+Attn] Epoch 003 - loss: 3524.5016 - mae: 47.4408 - mse: 3524.5016 - val_loss: 2824.9906 - val_mae: 44.0031 - val_mse: 2824.9906
[TCN+Attn] Epoch 004 - loss: 3545.4241 - mae: 47.5489 - mse: 3545.4241 - val_loss: 2779.6561 - val_mae: 44.2055 - val_mse: 2779.6561
[TCN+Attn] Epoch 005 - loss: 3407.1128 - mae: 46.6508 - mse: 3407.1128 - val_loss: 2275.9329 - val_mae: 39.3099 - val_mse: 2275.9329
[TCN+Attn] Epoch 006 - loss: 2033.4885 - mae: 33.3947 - mse: 2033.4885 - val_loss: 758.6242 - val_mae: 19.9114 - val_mse: 758.6242
[TCN+Attn] Epoch 007 - loss: 1509.6578 - mae: 27.3475 - mse: 1509.6578 - val_loss: 517.9582 - val_mae: 16.3629 - val_mse: 517.9582
[TCN+Attn] Epoch 008 - loss: 1138.9889 - mae: 23.2304 - mse: 1138.9889 - 

In [9]:
# %% [markdown]
# ## 7. Final RMSE Comparison

# %% [code]
def eval_model(model):
    model.eval(); preds=[]; targs=[]
    with torch.no_grad():
        for xb,yb in valid_loader:
            preds.append(model(xb.to(DEVICE)).cpu().numpy())
            targs.append(yb.numpy())
    return np.concatenate(preds), np.concatenate(targs)

preds_lstm, targ = eval_model(model_lstm)
preds_tf, _      = eval_model(model_tf)
preds_bi, _      = eval_model(model_bilstm)
preds_tc, _      = eval_model(model_tcn)

results = pd.DataFrame({
    'Model': ['LSTM','CNN-Transformer','BiLSTM+Attn','TCN+Attn'],
    'RMSE': [
        np.sqrt(mean_squared_error(targ, preds_lstm)),
        np.sqrt(mean_squared_error(targ, preds_tf)),
        np.sqrt(mean_squared_error(targ, preds_bi)),
        np.sqrt(mean_squared_error(targ, preds_tc)),
    ]
})
print("\nValidation RMSE Comparison:")
print(results)

# %% [markdown]
# **End of Notebook**  
# Adjust hyperparameters at the top and re-run sections to fine-tune.  


Validation RMSE Comparison:
             Model       RMSE
0             LSTM  19.708210
1  CNN-Transformer  20.839935
2      BiLSTM+Attn  20.496639
3         TCN+Attn  19.164568
